In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc,roc_curve
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook  import tqdm
import pickle
from scipy.sparse import hstack,csr_matrix
from sklearn.model_selection import train_test_split

In [50]:
def get_num_cat_cols_list(df):
    """get_num_cat_cols_list(df)... return num_cols,cat_cols
    This function returns the numerical columns list and categorical
    columns list of the passed dataframe"""
    num_cols=[]
    cat_cols=[]
    for i in df.columns:
        if (str(df[i].dtypes)[0:3]=="int")or(str(df[i].dtypes)[0:5]=="float"):
            num_cols.append(i)
        else:
            cat_cols.append(i)
    return num_cols,cat_cols
def create_agg_num(df,col_list):
    """create_agg_num(df,col_list) ... return df_new
    returns the aggregation of columns per client(SK_ID_CURR) - min, max,median,mean,count for
    numerical columns passed in col_list"""
    df_new=pd.DataFrame({})
    ids=[]
    cnt=0
    for j,i in tqdm(enumerate(col_list)):
        min_val=[]
        max_val=[]
        med_val=[]
        mean_val=[]
        counter_ids=0
        group_data=df.groupby(["SK_ID_CURR"])
        if j==0:
            ids=group_data[i].min().reset_index()
            df_new["SK_ID_CURR"]=ids["SK_ID_CURR"]
        df_temp=group_data.agg({i:["max","count"]}).reset_index()
        df_temp.columns=["SK_ID_CURR", str(i+"_MAX"),str(i+"_COUNT")]
        df_new=pd.concat([df_new,df_temp.drop(["SK_ID_CURR"],axis=1)],axis=1)
    return df_new

def get_na_count(df):
    """get_na_count(df)... return df_na_counts,col_na_rm
    This function returns a dataframe with column list of na values and percent na values.
    It also returns columns with more than 25percent na values."""
    col_na_rm=[]
    cols_with_na=[]
    count_na=[]
    percent_na=[]
    datatype_col=[]
    for i in df.columns:
        val=df[i].isna().sum()/df.shape[0]
        if val>=0.95:
            col_na_rm.append(i)
        elif 0.95>val>0:
            cols_with_na.append(i)
            count_na.append(df[i].isna().sum())
            percent_na.append(df[i].isna().sum()/df[i].shape[0])
            datatype_col.append(df[i].dtypes)
    df_na_counts=pd.DataFrame({"cols":cols_with_na,"na_count":count_na,"percent_na":percent_na,
                              "datatype_col":datatype_col})
    return df_na_counts,col_na_rm

def impute_na(impute_dict,df,na_col_list,flag=False,impute_value="",num_cols=[]):
    """impute_na(df,na_col_list,num_cols)....return df
    The function removes na values with replacing na with median for numeric column 
    and by mode for categorical column and returns the dataframe object with no NA's"""
    if not flag :
        for i in na_col_list:
            if i in num_cols:
                med=impute_dict[i]
                df.loc[df[i].isna()==True,i]=med
            else:
                df.loc[df[i].isna()==True,i]="norecord"
           
    else:
        for i in na_col_list:
            df.loc[df[i].isna()==True,i]=impute_value
    return df
def rm_outliers_other(df,col_list,bounds):
    for i in col_list:
        df.loc[df[i]>bounds[i][1],i]=bounds[i][1]
        df.loc[df[i]<bounds[i][0],i]=bounds[i][0]
    return df

def get_ohe_other_data(df,col_list,ohe_dict):
    cnt=0
    for i in col_list:
        ohe=ohe_dict[i]
        ohe_arr=ohe.transform(np.asarray(df[i]).reshape(-1,1))
        if cnt==0:
            arr=ohe_arr
        else:
            arr=hstack((arr,ohe_arr))
        cnt=1
    return arr

def get_scaled_data_other(df,col_list,scalers):
    cnt=0
    for i in tqdm(col_list):
        scaler=scalers[i]
        arr=scaler.transform(np.asarray(df[i]).reshape(-1,1))
        if cnt==0:
            mat_arr=arr
        else:
            mat_arr=np.column_stack((mat_arr,arr))
        cnt=1
    return mat_arr

In [41]:
def final_1(X):
    app_cols=X.columns
    X.drop([app_cols[5]],axis=1,inplace=True)
    X["INCOME2CREDIT"]=X[app_cols[6]]/X[app_cols[7]]
    X["ANNUITYTOCREDIT"]=X[app_cols[8]]/X[app_cols[7]]
    X["TOTINCTOCREDIT"]=(X[app_cols[6]]+X[app_cols[8]])/X[app_cols[7]]
    X.drop(app_cols[6:9],axis=1,inplace=True)
    X["Age"]=(-1*X[app_cols[16]])/365
    X["EXPERIENCE"]=X[app_cols[17]]/365
    X["REGYEARS"]=(-1*X[app_cols[18]])/365
    X["PUBYEARS"]=(-1*X[app_cols[19]])/365
    X["CONTACTINFOCNT"]=X[app_cols[21]]+X[app_cols[22]]+X[app_cols[23]]+X[app_cols[24]]+X[app_cols[25]]+X[app_cols[26]]
    X.drop(app_cols[21:27],axis=1,inplace=True)
    X["CONTACTINFOCNT"]=X["CONTACTINFOCNT"].astype("object")
    X[app_cols[29]]=X[app_cols[29]].astype("object")
    X[app_cols[30]]=X[app_cols[30]].astype("object")
    X[app_cols[32]]=X[app_cols[32]].astype("object")
    X["GRPAPT_MEDI"]=X["APARTMENTS_MEDI"]+X["ELEVATORS_MEDI"]+X["LIVINGAPARTMENTS_MEDI"]+X["LIVINGAREA_MEDI"]
    X.drop(["APARTMENTS_MEDI","ELEVATORS_MEDI","LIVINGAPARTMENTS_MEDI","LIVINGAREA_MEDI"],axis=1,inplace=True)
    del_cols=[i for i in app_cols[43:90] if "_MEDI" not in i]
    X.drop(del_cols,axis=1,inplace=True)
    X["PHONECHANGEDYRS"]=(-1*X[app_cols[94]])/365
    X["DOCCNT"]=X[app_cols[95]]+X[app_cols[96]]+X[app_cols[97]]+X[app_cols[98]]+X[app_cols[99]]+\
          X[app_cols[100]]+X[app_cols[101]]+X[app_cols[102]]+X[app_cols[103]]+X[app_cols[104]]+\
          X[app_cols[105]]+X[app_cols[106]]+X[app_cols[107]]+X[app_cols[108]]+X[app_cols[109]]+\
          X[app_cols[110]]+X[app_cols[111]]+X[app_cols[112]]+X[app_cols[113]]+X[app_cols[114]]
    X.drop(app_cols[95:115],axis=1,inplace=True)
    X.drop(["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","DAYS_LAST_PHONE_CHANGE"],axis=1,inplace=True)
    X["LIVE_CITY_NOT_WORK_CITY"]=X["LIVE_CITY_NOT_WORK_CITY"].astype("object")
    X["REG_CITY_NOT_LIVE_CITY"]=X.REG_CITY_NOT_LIVE_CITY.astype("object")
    X["REG_CITY_NOT_WORK_CITY"]=X.REG_CITY_NOT_WORK_CITY.astype("object")
    X["LIVE_REGION_NOT_WORK_REGION"]=X.LIVE_REGION_NOT_WORK_REGION.astype("object")
    X["REG_REGION_NOT_LIVE_REGION"]=X.REG_REGION_NOT_LIVE_REGION.astype("object")
    X["REG_REGION_NOT_LIVE_REGION"]=X.REG_REGION_NOT_LIVE_REGION.astype("object")
    X.drop(["OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE","INCOME2CREDIT"],axis=1,inplace=True)
    
    bureau=pd.read_csv("bureau.csv")
    num_cols,_=get_num_cat_cols_list(bureau)
    bureau_agg=create_agg_num(bureau,num_cols[2:])
    bureau_agg["BUREAU_CNT"]=bureau_agg["DAYS_CREDIT_COUNT"]
    del_cols=[i for i in bureau_agg.columns[1:-1] if "COUNT" in i]
    bureau_agg.drop(del_cols,axis=1,inplace=True)
    del bureau
    X=pd.merge(X,bureau_agg,how="left",on="SK_ID_CURR")
    del bureau_agg
    
    
    install=pd.read_csv("installments_payments.csv")
    install_cols=install.columns
    install["INSTALMENT_YRS"]=(-1*install[install_cols[4]])/365
    install["YRS_ENTRY_PAYMENT"]=(-1*install[install_cols[5]])/365
    install.drop([install_cols[4],install_cols[5]],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(install)
    install_agg=create_agg_num(install,num_cols[2:])
    del install
    install_agg["INSTALL_CNT"]=install_agg.NUM_INSTALMENT_VERSION_COUNT
    del_cols=[i for i in install_agg.columns if "COUNT" in i]
    install_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,install_agg,how="left",on="SK_ID_CURR")
    
    
    pos_cash=pd.read_csv("POS_CASH_balance.csv")
    pos_cash.drop(["MONTHS_BALANCE"],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(pos_cash)
    pos_cash_agg=create_agg_num(pos_cash,num_cols[2:])
    pos_cash_agg["POS_CNT"]=pos_cash_agg.CNT_INSTALMENT_COUNT
    del_cols=[i for i in pos_cash_agg.columns if "COUNT" in i]
    del pos_cash
    pos_cash_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,pos_cash_agg,how="left",on="SK_ID_CURR")
    del pos_cash_agg
    
    
    cc_bal=pd.read_csv("credit_card_balance.csv")
    cc_bal.drop(["MONTHS_BALANCE"],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(cc_bal)
    cc_bal_agg=create_agg_num(cc_bal,num_cols[2:])
    del cc_bal
    cc_bal_agg["CC_CNT"]=cc_bal_agg.AMT_RECEIVABLE_PRINCIPAL_COUNT
    del_cols=[i for i in cc_bal_agg.columns if "COUNT" in i]
    cc_bal_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,cc_bal_agg,how="left",on="SK_ID_CURR")
    del cc_bal_agg
    
    
    prev_app=pd.read_csv("previous_application.csv")
    num_cols,_=get_num_cat_cols_list(prev_app)
    prev_app_agg=create_agg_num(prev_app,num_cols[2:])
    prev_app_agg["PREV_CNT"]=prev_app_agg.AMT_ANNUITY_COUNT
    del_cols=[i for i in prev_app_agg.columns if "COUNT" in i]
    prev_app_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,prev_app_agg,how="left",on="SK_ID_CURR")
    del prev_app_agg
    del prev_app
    
    
    num_cols,cat_cols=get_num_cat_cols_list(X)
    bound_dict,imputation_dict=pickle.load(open("Final_model_files/outliers_na_impute_dicts.pkl","rb"))
    X=rm_outliers_other(X,num_cols[1:],bound_dict)
    na_info,col_rm=get_na_count(X)
    X=impute_na(imputation_dict,X.copy(),na_info.cols[na_info.datatype_col=="object"],False,"",num_cols)
    X=impute_na(imputation_dict,X.copy(),na_info.cols[na_info.datatype_col!="object"],False,"",num_cols)
    
    
    X.drop(["REGION_RATING_CLIENT_W_CITY"],axis=1,inplace=True)
    
    num_cols,cat_cols=get_num_cat_cols_list(X)
    ohe_list,scaler_list=pickle.load(open("Final_model_files/ohe_scaler_list.pkl","rb"))
    X_ohe_arr=get_ohe_other_data(X,cat_cols,ohe_list)
    X_num=get_scaled_data_other(X,num_cols[1:],scaler_list)
    X_new=hstack((X_ohe_arr,X_num))
    keep_cols=list(set(range(269))-set([257,258]))
    X_new=csr_matrix(X_new.todense()[:,keep_cols])
    
    bst=pickle.load(open("Final_model_files/Final_model.pkl","rb"))
    y_pred=bst.predict(X_new)
    
    return y_pred

In [34]:
def final_2(X,Y):
    app_cols=X.columns

    X.drop([app_cols[5]],axis=1,inplace=True)
    X["INCOME2CREDIT"]=X[app_cols[6]]/X[app_cols[7]]
    X["ANNUITYTOCREDIT"]=X[app_cols[8]]/X[app_cols[7]]
    X["TOTINCTOCREDIT"]=(X[app_cols[6]]+X[app_cols[8]])/X[app_cols[7]]
    X.drop(app_cols[6:9],axis=1,inplace=True)
    X["Age"]=(-1*X[app_cols[16]])/365
    X["EXPERIENCE"]=X[app_cols[17]]/365
    X["REGYEARS"]=(-1*X[app_cols[18]])/365
    X["PUBYEARS"]=(-1*X[app_cols[19]])/365
    X["CONTACTINFOCNT"]=X[app_cols[21]]+X[app_cols[22]]+X[app_cols[23]]+X[app_cols[24]]+X[app_cols[25]]+X[app_cols[26]]
    X.drop(app_cols[21:27],axis=1,inplace=True)
    X["CONTACTINFOCNT"]=X["CONTACTINFOCNT"].astype("object")
    X[app_cols[29]]=X[app_cols[29]].astype("object")
    X[app_cols[30]]=X[app_cols[30]].astype("object")
    X[app_cols[32]]=X[app_cols[32]].astype("object")
    X["GRPAPT_MEDI"]=X["APARTMENTS_MEDI"]+X["ELEVATORS_MEDI"]+X["LIVINGAPARTMENTS_MEDI"]+X["LIVINGAREA_MEDI"]
    X.drop(["APARTMENTS_MEDI","ELEVATORS_MEDI","LIVINGAPARTMENTS_MEDI","LIVINGAREA_MEDI"],axis=1,inplace=True)
    del_cols=[i for i in app_cols[43:90] if "_MEDI" not in i]
    X.drop(del_cols,axis=1,inplace=True)
    X["PHONECHANGEDYRS"]=(-1*X[app_cols[94]])/365
    X["DOCCNT"]=X[app_cols[95]]+X[app_cols[96]]+X[app_cols[97]]+X[app_cols[98]]+X[app_cols[99]]+\
          X[app_cols[100]]+X[app_cols[101]]+X[app_cols[102]]+X[app_cols[103]]+X[app_cols[104]]+\
          X[app_cols[105]]+X[app_cols[106]]+X[app_cols[107]]+X[app_cols[108]]+X[app_cols[109]]+\
          X[app_cols[110]]+X[app_cols[111]]+X[app_cols[112]]+X[app_cols[113]]+X[app_cols[114]]
    X.drop(app_cols[95:115],axis=1,inplace=True)
    X.drop(["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","DAYS_LAST_PHONE_CHANGE"],axis=1,inplace=True)
    X["LIVE_CITY_NOT_WORK_CITY"]=X["LIVE_CITY_NOT_WORK_CITY"].astype("object")
    X["REG_CITY_NOT_LIVE_CITY"]=X.REG_CITY_NOT_LIVE_CITY.astype("object")
    X["REG_CITY_NOT_WORK_CITY"]=X.REG_CITY_NOT_WORK_CITY.astype("object")
    X["LIVE_REGION_NOT_WORK_REGION"]=X.LIVE_REGION_NOT_WORK_REGION.astype("object")
    X["REG_REGION_NOT_LIVE_REGION"]=X.REG_REGION_NOT_LIVE_REGION.astype("object")
    X["REG_REGION_NOT_LIVE_REGION"]=X.REG_REGION_NOT_LIVE_REGION.astype("object")
    X.drop(["OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE","INCOME2CREDIT"],axis=1,inplace=True)
    
    bureau=pd.read_csv("bureau.csv")
    num_cols,_=get_num_cat_cols_list(bureau)
    bureau_agg=create_agg_num(bureau,num_cols[2:])
    bureau_agg["BUREAU_CNT"]=bureau_agg["DAYS_CREDIT_COUNT"]
    del_cols=[i for i in bureau_agg.columns[1:-1] if "COUNT" in i]
    bureau_agg.drop(del_cols,axis=1,inplace=True)
    del bureau
    X=pd.merge(X,bureau_agg,how="left",on="SK_ID_CURR")
    del bureau_agg
    
    
    install=pd.read_csv("installments_payments.csv")
    install_cols=install.columns
    install["INSTALMENT_YRS"]=(-1*install[install_cols[4]])/365
    install["YRS_ENTRY_PAYMENT"]=(-1*install[install_cols[5]])/365
    install.drop([install_cols[4],install_cols[5]],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(install)
    install_agg=create_agg_num(install,num_cols[2:])
    del install
    install_agg["INSTALL_CNT"]=install_agg.NUM_INSTALMENT_VERSION_COUNT
    del_cols=[i for i in install_agg.columns if "COUNT" in i]
    install_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,install_agg,how="left",on="SK_ID_CURR")
    
    
    pos_cash=pd.read_csv("POS_CASH_balance.csv")
    pos_cash.drop(["MONTHS_BALANCE"],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(pos_cash)
    pos_cash_agg=create_agg_num(pos_cash,num_cols[2:])
    pos_cash_agg["POS_CNT"]=pos_cash_agg.CNT_INSTALMENT_COUNT
    del_cols=[i for i in pos_cash_agg.columns if "COUNT" in i]
    del pos_cash
    pos_cash_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,pos_cash_agg,how="left",on="SK_ID_CURR")
    del pos_cash_agg
    
    
    cc_bal=pd.read_csv("credit_card_balance.csv")
    cc_bal.drop(["MONTHS_BALANCE"],axis=1,inplace=True)
    num_cols,_=get_num_cat_cols_list(cc_bal)
    cc_bal_agg=create_agg_num(cc_bal,num_cols[2:])
    del cc_bal
    cc_bal_agg["CC_CNT"]=cc_bal_agg.AMT_RECEIVABLE_PRINCIPAL_COUNT
    del_cols=[i for i in cc_bal_agg.columns if "COUNT" in i]
    cc_bal_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,cc_bal_agg,how="left",on="SK_ID_CURR")
    del cc_bal_agg
    
    
    prev_app=pd.read_csv("previous_application.csv")
    num_cols,_=get_num_cat_cols_list(prev_app)
    prev_app_agg=create_agg_num(prev_app,num_cols[2:])
    prev_app_agg["PREV_CNT"]=prev_app_agg.AMT_ANNUITY_COUNT
    del_cols=[i for i in prev_app_agg.columns if "COUNT" in i]
    prev_app_agg.drop(del_cols,axis=1,inplace=True)
    X=pd.merge(X,prev_app_agg,how="left",on="SK_ID_CURR")
    del prev_app_agg
    del prev_app
    
    
    num_cols,cat_cols=get_num_cat_cols_list(X)
    bound_dict,imputation_dict=pickle.load(open("Final_model_files/outliers_na_impute_dicts.pkl","rb"))
    X=rm_outliers_other(X,num_cols[1:],bound_dict)
    na_info,col_rm=get_na_count(X)
    X=impute_na(imputation_dict,X.copy(),na_info.cols[na_info.datatype_col=="object"],False,"",num_cols)
    X=impute_na(imputation_dict,X.copy(),na_info.cols[na_info.datatype_col!="object"],False,"",num_cols)
    
    
    X.drop(["REGION_RATING_CLIENT_W_CITY"],axis=1,inplace=True)
    
    num_cols,cat_cols=get_num_cat_cols_list(X)
    ohe_list,scaler_list=pickle.load(open("Final_model_files/ohe_scaler_list.pkl","rb"))
    X_ohe_arr=get_ohe_other_data(X,cat_cols,ohe_list)
    X_num=get_scaled_data_other(X,num_cols[1:],scaler_list)
    X_new=hstack((X_ohe_arr,X_num))
    keep_cols=list(set(range(269))-set([257,258]))
    X_new=csr_matrix(X_new.todense()[:,keep_cols])
    
    Y=np.asarray(Y)
    
    bst=pickle.load(open("Final_model_files/Final_model.pkl","rb"))
    y_pred=bst.predict(X_new)
    fpr, tpr, thresholds = roc_curve(Y,y_pred,  pos_label=1)
    metric_value=auc(fpr, tpr)
    
    return metric_value

In [3]:
!gdown --id 1LO3ebiwGk1EXVwsn9K-jtmynUyHUkDUP
!unzip Final_model_files.zip

Downloading...
From: https://drive.google.com/uc?id=1LO3ebiwGk1EXVwsn9K-jtmynUyHUkDUP
To: /content/Final_model_files.zip
100% 110k/110k [00:00<00:00, 31.8MB/s]
Archive:  Final_model_files.zip
  inflating: Final_model_files/Final_model.pkl  
  inflating: Final_model_files/ohe_scaler_list.pkl  
  inflating: Final_model_files/outliers_na_impute_dicts.pkl  


In [6]:
!gdown --id 1xnQWNBTKut2nkGeLF9BDMu2scuaiw745
!unzip home-credit-default-risk.zip

Downloading...
From: https://drive.google.com/uc?id=1xnQWNBTKut2nkGeLF9BDMu2scuaiw745
To: /content/home-credit-default-risk.zip
100% 722M/722M [00:08<00:00, 88.3MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [32]:
#reading file to test the created two functions
train=pd.read_csv("application_train.csv")
x=train.drop(["TARGET"],axis=1)
y=train.TARGET
train,test,y_train,y_test=train_test_split(x,y,test_size=0.30,stratify=y,random_state=100)

In [51]:
#Getting auc for test set
auc_metric=final_2(test.copy(),y_test)
print("AUC: ",auc_metric)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

AUC:  0.7731455887707113


In [52]:
#getting y_pred for test set
y_pred=final_1(test.copy())

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [53]:
#Checking predicted output against actuals
y=np.asarray(y_test)
y=pd.Series(y)
y_all=pd.concat([y,pd.Series(y_pred)],axis=1)
y_all.head(4)

,0,1
0,1,0.729388
1,0,0.261724
2,0,0.406862
3,0,0.178205
